In [11]:
import pandas as pd
import json

import requests
import xml.etree.ElementTree as ET
from urllib.parse import quote

from geopy.geocoders import Nominatim
import folium

import time

### (1) Step 1: 

Create JSON file with all state capitals and their addresses.

Ask claude: Search for the capital & zipcode for 50 states in US


### Step 2: 
Verify that its a valid JSON file  that you created

https://jsonlint.com/

https://codebeautify.org/jsonvalidator

In [12]:
with open('us_state_capitals.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

#### Step 2.1: 
Verify addresses.using USPS API.

https://tools.usps.com/zip-code-lookup.htm

https://www.census.gov/data.html

In [16]:
geolocator = Nominatim(user_agent="address_verifier")
ls_verify = []
ls_zip = []

for i, row in df.iterrows():
    street = row['address']['street']
    city = row['address']['city']
    state = row['address']['state_code']
    zip5 = row['address']['zip']
    query = f"{street}, {city}, {state}"

    try:
        location = geolocator.geocode(query, country_codes="us")
        if location:
            # print(f"Address found: {row['state']}")
            ls_verify.append("Address found")
            
        else:
            print(f"Address not found: {row['state']}")
            ls_verify.append("Address not found")
    except:
        print(f"Error processing address: {row['state']}")
        ls_verify.append("Error processing address")

    time.sleep(10)  # To avoid hitting the API too quickly

Error processing address: Illinois
Error processing address: Minnesota
Address not found: New York
Address not found: Ohio
Error processing address: Tennessee
Error processing address: Utah


In [17]:
df['valid address?'] = ls_verify

### Step 3: 
Write a python program that reads that JSON file and creates another JSON file with the same things plus Longitude and latitude.


In [18]:
latitudes = []
longitudes = []

geolocator = Nominatim(user_agent="geoapi")

for _, row in df.iterrows():
    try:
        location = geolocator.geocode(f"{row['capital']}, {row['state']}, USA")

        try:
            latitude = location.latitude
            longitude = location.longitude
            # print(f"{capital}: ({location.latitude}, {location.longitude})")
        except:
            print(f"Error retrieving coordinates for {row['capital']}")
            latitude = eval(input("Enter latitude of {}: ".format(row['capital'])))
            longitude = eval(input("Enter longitude of {}: ".format(row['capital'])))
    except:
        print(f"{row['capital']}: Not found")
        latitude = eval(input("Enter latitude of {}: ".format(row['capital'])))
        longitude = eval(input("Enter longitude of {}: ".format(row['capital'])))

    latitudes.append(latitude)
    longitudes.append(longitude)
    time.sleep(10)

In [19]:
df['latitude'] = latitudes
df['longitude'] = list(longitudes)

In [20]:
df.to_json('us_state_capitals_with_coordinates.json', orient='records',indent=4,index=False)

### Step 4: 
Verify that the Longitude and latitude are variable.

website: https://www.google.com/maps?q=38.8977,-77.0365 (change the numbers)

In [21]:
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

In [22]:
for _, row in df.iterrows():

    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

    folium.map.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'''<div style="font-size: 10pt; color: black;">{row['state']}<br>{row['capital']}</div>'''
        )
    ).add_to(m)

In [23]:
m.save("us_capitals_check.html")

### Step 5: 
Verify the JSON file again.


In [24]:
with open('us_state_capitals_with_coordinates.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)